In [9]:
import json
import shutil
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from pyproj import Transformer
from sklearn.cluster import DBSCAN

import rasterio
from rasterio.merge import merge
from rasterio.shutil import copy
from rasterio.enums import Resampling

RAW_PRED_FOLDER = Path("./data/PREDICTIONS_RASTER")
MERGED_RASTERS = Path("./data/merged_rasters")
COLORED_RASTERS = Path("./data/colored_rasters")

In [ ]:
# The get_path function try to match the session_name with a session on your computer.
# Please, change the path to your local folder.


def get_path(row):
    name = row["session_name"]

    date = name.split("_")[0][0:6]


    letter_disk = ""
    if date in ["202312", "202311"]:
        letter_disk = "D"
    elif date in ["202211", "202309", "202310"]:
        letter_disk = "E"
    else:
        letter_disk = "F"

    if 1 <= int(date[5]) <= 7 and int(date[3]) == 3 and int(date[4]) == 0 and "MDG" not in name:
        date = "202301-07"

    return f"/media/bioeos/{letter_disk}/{date}_plancha_session"


# Working from file : https://zenodo.org/records/16924962/files/session_doi.csv?download=1
df = pd.read_csv("session_doi.csv")
df = df[df['session_name'].str.contains('ASV', na=False)].reset_index()
df["root_folder"] = df.apply(lambda row: get_path(row), axis=1)
df = df[["root_folder", "session_name"]]

# Verify if all 
for i, row in df.iterrows():
    session_path = Path(row["root_folder"], row["session_name"])
    if not session_path.exists():
        print(session_path)

df.to_csv("session_asv.csv", index=False)

Use this script to scrap all IA files : https://github.com/SeatizenDOI/plancha-workflow/blob/master/utils/master_grab.py

with this command : python master_grab.py -ecsv -pcsv /home/bioeos/Documents/project_hub/cog-server/tools/pred_asv/session_asv.csv -po /home/bioeos/Documents/project_hub/cog-server/tools/pred_asv/data -pr

In [2]:
# Now we remove the useless class

USEFUL_CLASS = ["ACROPORE_BRANCHED", "ACROPORE_DIGITISED", "ACROPORE_SUB_MASSIVE", "ACROPORE_TABULAR", "ALGAE_ASSEMBLY", "ALGAE_DRAWN_UP", "ALGAE_LIMESTONE", "ALGAE_SODDING", "BLEACHED_CORAL", "DEAD_CORAL", "MILLEPORE", "NO_ACROPORE_ENCRUSTING", "NO_ACROPORE_FOLIACEOUS", "NO_ACROPORE_MASSIVE", "NO_ACROPORE_SOLITARY", "NO_ACROPORE_SUB_MASSIVE", "ROCK", "RUBBLE", "SAND", "SYRINGODIUM_ISOETIFOLIUM", "THALASSODENDRON_CILIATUM"]

for folder in sorted(list(Path(RAW_PRED_FOLDER).iterdir())):
    if folder.name in USEFUL_CLASS: continue

    print(f"Removing {folder.name}")
    shutil.rmtree(folder)
    # print(f'"{folder.name}"', end=", ") # Use this line to show in list format all tags

In [3]:
# This cell gather all functions necessary to produce the merged raster.

def get_raster_bbox(raster_path):
    """Extract the bounding box of a raster file in EPSG:3857 (meters)."""
    with rasterio.open(raster_path) as dataset:
        bounds = dataset.bounds  # (left, bottom, right, top)

        # Convert coordinates to meters (EPSG:4326 to EPSG:3857)
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
        min_x, min_y = transformer.transform(bounds.left, bounds.bottom)
        max_x, max_y = transformer.transform(bounds.right, bounds.top)

        return min_x, min_y, max_x, max_y, raster_path

def bbox_distance(bbox1, bbox2):
    """Compute the minimum distance between two bounding boxes."""
    min_x1, min_y1, max_x1, max_y1, _ = bbox1
    min_x2, min_y2, max_x2, max_y2, _ = bbox2

    # Compute horizontal and vertical distances
    dx = max(min_x2 - max_x1, min_x1 - max_x2, 0)  # Distance along x-axis
    dy = max(min_y2 - max_y1, min_y1 - max_y2, 0)  # Distance along y-axis

    return np.hypot(dx, dy)  # Euclidean distance

def group_rasters_by_bbox(raster_paths, distance_threshold=10):
    """Groups raster files based on bounding box proximity."""
    bboxes = [get_raster_bbox(path) for path in raster_paths]

    # Compute pairwise distance matrix
    distance_matrix = np.zeros((len(bboxes), len(bboxes)))
    for i in range(len(bboxes)):
        for j in range(len(bboxes)):
            if i != j:
                distance_matrix[i, j] = bbox_distance(bboxes[i], bboxes[j])

    # Use DBSCAN clustering
    clustering = DBSCAN(eps=distance_threshold, min_samples=1, metric="precomputed").fit(distance_matrix)

    # Organize rasters by cluster labels
    grouped_rasters = {}
    for bbox, label in zip(bboxes, clustering.labels_):
        if label not in grouped_rasters:
            grouped_rasters[label] = []
        grouped_rasters[label].append(bbox)

    return list(grouped_rasters.values())

def merge_rasters_to_cog(groups, year, species, output_folder):
    """Merge each group into a single COG file."""
    output_folder = Path(output_folder)
    output_folder.mkdir(exist_ok=True)

    for i, group in enumerate(groups):
        raster_paths = [bbox[4] for bbox in group]  # Extract file paths
        # print(f"Generate {i}")

        # Open all rasters
        datasets = [rasterio.open(path) for path in raster_paths]


        # Merge the rasters
        mosaic, out_transform = merge(datasets, resampling=Resampling.bilinear)

        # Use metadata from first raster
        out_meta = datasets[0].meta.copy()
        out_meta.update({
            "driver": "COG",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform,
            "compress": "LZW",
            "BIGTIFF": "IF_SAFER"
        })

        merged_path = Path(output_folder, f"tmp_{i}.tif")

        # Save the merged raster
        with rasterio.open(merged_path, "w", **out_meta) as dest:
            dest.write(mosaic)

        # Convert to COG
        cog_path = Path(output_folder, f"group_{year}_{species}_{i}.tif")
        copy(merged_path, cog_path, driver="COG", compress="LZW")

        merged_path.unlink()

        # Close datasets
        for dataset in datasets:
            dataset.close()

In [4]:
years = list(set([a.name[0:4] for a in Path(RAW_PRED_FOLDER, "SAND").iterdir()]))

print(f"Years found {sorted(years)}")

# Clean output folder.
if MERGED_RASTERS.exists():
    shutil.rmtree(MERGED_RASTERS)


for species_folder in tqdm(sorted(list(RAW_PRED_FOLDER.iterdir()))):
    for year in years:
        print(f"Working with year {year}")
        raster_files = sorted([a for a in Path(species_folder).iterdir() if year in a.name])

        groups = group_rasters_by_bbox(raster_files, distance_threshold=10)

        output_folder = Path(MERGED_RASTERS, year)
        output_folder.mkdir(exist_ok=True, parents=True)

        cog_files = merge_rasters_to_cog(groups, year, species_folder.name, output_folder)



Years found ['2022', '2023', '2024', '2025']


  0%|          | 0/22 [00:00<?, ?it/s]

Working with year 2024
Working with year 2025
Working with year 2022
Working with year 2023


  5%|▍         | 1/22 [00:04<01:35,  4.55s/it]

Working with year 2024
Working with year 2025
Working with year 2022
Working with year 2023


...

 91%|█████████ | 20/22 [01:28<00:08,  4.26s/it]

Working with year 2024
Working with year 2025
Working with year 2022
Working with year 2023


 95%|█████████▌| 21/22 [01:32<00:04,  4.37s/it]

Working with year 2024
Working with year 2025
Working with year 2022
Working with year 2023


100%|██████████| 22/22 [01:37<00:00,  4.43s/it]


In [11]:
thresholds = {"Acropore_branched": 0.351, "Acropore_digitised": 0.349, "Acropore_sub_massive": 0.123, "Acropore_tabular": 0.415, "Algae_assembly": 0.434, "Algae_drawn_up": 0.193, "Algae_limestone": 0.346, "Algae_sodding": 0.41, "Atra/Leucospilota": 0.586, "Bleached_coral": 0.408, "Blurred": 0.3, "Dead_coral": 0.407, "Fish": 0.466, "Homo_sapiens": 0.402, "Human_object": 0.343, "Living_coral": 0.208, "Millepore": 0.292, "No_acropore_encrusting": 0.227, "No_acropore_foliaceous": 0.462, "No_acropore_massive": 0.333, "No_acropore_solitary": 0.415, "No_acropore_sub_massive": 0.377, "Rock": 0.476, "Sand": 0.548, "Rubble": 0.417, "Sea_cucumber": 0.357, "Sea_urchins": 0.335, "Sponge": 0.152, "Syringodium_isoetifolium": 0.476, "Thalassodendron_ciliatum": 0.209, "Useless": 0.315}
thresholds = {k.upper():v for k, v in thresholds.items()}


color_map = {k.upper(): np.array([random.randint(100, 255),  # R
                        random.randint(100, 255),  # G
                        random.randint(100, 255)]) for k in thresholds}
color_map_to_save = {k: v.tolist() for k,v in color_map.items()}

with open('color_asv_pred_by_specie.json', 'w', encoding='utf-8') as f:
    json.dump(color_map_to_save, f, indent=4)

In [12]:

def create_raster_color(raster_path, output_tif, threshold, base_color):

    with rasterio.open(raster_path) as src:
        band = src.read(1)
        profile = src.profile

        # Replace NaN with 0 for safe math
        band_clean = np.where(np.isnan(band), 0, band)

        # Keep values in [0,1] range
        band_norm = np.clip(band_clean, 0, 1)

        # Mask: true if pixel should be transparent (below threshold OR NaN)
        mask = (band_norm < threshold) | np.isnan(band)

        # Build RGBA
        rgba = np.zeros((4, band.shape[0], band.shape[1]), dtype=np.uint8)

        # Blend from white → base_color
        for c in range(3):  # R,G,B
            rgba[c] = (255 * (1 - band_norm) + base_color[c] * band_norm).astype(np.uint8)
        rgba[3] = 255  # default opaque
        rgba[:, mask] = 0  # all channels 0 if transparent

        # Update profile for RGBA with explicit alpha
        profile.update(
            count=4,
            dtype="uint8",
            photometric="RGBA",
            compress="LZW",
            nodata=None
        )
        profile.pop("nodata", None)

        with rasterio.open(output_tif, "w", **profile) as dst:
            dst.write(rgba)
            dst.set_band_description(1, "Red")
            dst.set_band_description(2, "Green")
            dst.set_band_description(3, "Blue")
            dst.set_band_description(4, "Alpha")  # mark alpha band
            dst.colorinterp = (
                rasterio.enums.ColorInterp.red,
                rasterio.enums.ColorInterp.green,
                rasterio.enums.ColorInterp.blue,
                rasterio.enums.ColorInterp.alpha
            )

In [13]:

# Clean output folder.
if COLORED_RASTERS.exists():
    shutil.rmtree(COLORED_RASTERS)


for year in sorted(list(MERGED_RASTERS.iterdir())):
    print(year)
    for raster in tqdm(sorted(list(year.iterdir()))):

        year_output = Path(COLORED_RASTERS, year.name)
        year_output.mkdir(exist_ok=True, parents=True)
        
        output_file = Path(year_output, f"{raster.stem}_colored.tif")

        name_split = raster.name.split("_")
        species = "_".join(name_split[2:len(name_split)-1])
        threshold = thresholds.get(species, 0.5)
        color = color_map.get(species, (255, 100, 100))
        
        create_raster_color(raster, output_file, threshold, color)


data/merged_rasters/2022


100%|██████████| 264/264 [00:05<00:00, 46.34it/s]


data/merged_rasters/2023


100%|██████████| 528/528 [00:10<00:00, 51.82it/s]


data/merged_rasters/2024


100%|██████████| 154/154 [00:03<00:00, 48.18it/s]


data/merged_rasters/2025


100%|██████████| 132/132 [00:02<00:00, 56.07it/s]
